## 9.6 Deutsch algorithm (part 1)
This code shows how 4 different Oracles, representing 4 classical functions, act when being applied to different input states.

In [ ]:
using ImageShow
using StrangelyDisplayed
using StrangelyQuantum

The following function constructs one of four different oracles, depending on the input argument `f`.

In [ ]:
function createOracle(f)
    matrix = zeros(ComplexF64, 4, 4)

    if f == 1
        matrix[1, 1] = 1
        matrix[2, 2] = 1
        matrix[3, 3] = 1
        matrix[4, 4] = 1
        return Oracle(matrix)
    elseif f == 2
        matrix[1, 1] = 1
        matrix[2, 2] = 1
        matrix[3, 4] = 1
        matrix[4, 3] = 1
        return Oracle(matrix)
    elseif f == 3
        matrix[1, 2] = 1
        matrix[2, 1] = 1
        matrix[3, 3] = 1
        matrix[4, 4] = 1
        return Oracle(matrix)
    elseif f == 4
        matrix[1, 2] = 1
        matrix[2, 1] = 1
        matrix[3, 4] = 1
        matrix[4, 3] = 1
        return Oracle(matrix)
    else
        throw("Wrong index in oracle construction")
    end
end

The next function constructs a program with two qubits, whos initial values are given by `qubit1` and `qubit2` respectively. The third argument selects an oracle (using `createOracle`), which is applied to both qubits.

In [ ]:
function construct_oracle_program(qubit1, qubit2, choice)
    program = Program(2)

    prepareStep = Step()
    qubit1 == 1 && addGate(prepareStep, X(1))
    qubit2 == 2 && addGate(prepareStep, X(2))
    addStep(program, prepareStep)

    oracleStep = Step()
    oracle = createOracle(choice)
    addGate(oracleStep, oracle)
    addStep(program, oracleStep)
    return program
end

In [ ]:
One example program (and its sampling histogram) is displayed here:

In [ ]:
sample = construct_oracle_program(0, 0, 2)

In [ ]:
drawProgram(sample)

In [ ]:
drawTrialHistogram(sample, 1000)

The next function uses `construct_oracle_program` to construct a program using the provided initial values for the two qubits, and cycles through all available oracles tp determine the results.

In [ ]:
function apply_oracle(qubit1, qubit2)
    simulator = SimpleQuantumExecutionEnvironment()
    for choice = 1:4
        program = construct_oracle_program(qubit1, qubit2, choice)

        result = runProgram(simulator, program)
        qubits = getQubits(result)

        constant = choice == 1 || choice == 4

        println(constant ? "C" : "B", ", measured = |",
                measure(qubits[1]), measure(qubits[2]), ">")
    end
end

Now a function to use all posible pairs of input values:

In [ ]:
function oracle_example2()
    println("Use 00 as input")
    apply_oracle(0, 0)
    println("\nUse 01 as input")
    apply_oracle(0, 1)
    println("\nUse 10 as input")
    apply_oracle(1, 0)
    println("\nUse 11 as input")
    apply_oracle(1, 1)
end

Running it gives the following:

In [ ]:
oracle_example2()

All output values are either $|00\hspace{-0.25em}>$ or $|10\hspace{-0.25em}>$, but no combination of inputs allows us to distinguish between the constant functions (marked $C$) and the balanced functions (marked $B$).

A [different program](ch09-05-deutsch.ipynb) is required for this.